In [25]:
from cassandra.cluster import Cluster
import csv
import pandas as pd

In [20]:
cluster = Cluster(['localhost'])  # Replace with your Cassandra instance address
session = cluster.connect('nirf')  # Replace with your keyspace name

In [21]:
insert_stmt = session.prepare("""
    INSERT INTO nirf_score_by_faculty (
        faculty_id, faculty_name, year, rank, aggregated_score, 
        teaching_learning_and_resources, research_and_professional_practice, 
        graduation_outcomes, outreach_and_inclusivity, peer_perception
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")


In [36]:
with open('/Users/omersegal/University/Year 3/Semester 1/Big Data Platform/Homework/Exercise 1/NIRF-Rankings/Dataset/2016/EngineeringRanking_2016.csv', 'r') as csvfile:  # Replace with your dataset file
    reader = csv.reader(csvfile)
    next(reader, None)  # Skip the header row
    for row in reader:
        session.execute(insert_stmt, (row[0], row[1], 2016, int(row[5]), float(row[4]), float(row[6]), float(row[7]), float(row[8]), float(row[9]), float(row[10])))

In [43]:
# make path relative and not absolute (for ingestion)
# add external for loop to iterate across years

In [62]:
cluster.shutdown()

In [59]:
result_set = session.execute("""
    SELECT * FROM nirf_score_by_faculty;
""")
df = pd.DataFrame(list(result_set))
df = df.round(2)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your dataframe is named df

# List of score columns to compare with peer_perception
score_columns = ['teaching_learning_and_resources', 'research_and_professional_practice', 'graduation_outcomes', 'outreach_and_inclusivity']

# Scatter plots
for col in score_columns:
    sns.scatterplot(data=df, x='peer_perception', y=col)
    plt.title(f'Peer Perception vs {col}')
    plt.xlabel('Peer Perception')
    plt.ylabel(col)
    plt.show()

# Correlation heatmap
correlation_matrix = df[['peer_perception'] + score_columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


## Database Schema

In [ ]:
CREATE KEYSPACE IF NOT EXISTS nirf WITH replication = {
    'class': 'SimpleStrategy',
    'replication_factor': 1
};

In [ ]:
DESCRIBE keyspaces;

In [ ]:
USE nirf;

In [ ]:
CREATE TABLE nirf_score_by_faculty (
    faculty_id TEXT,
    faculty_name TEXT,
    year int,
    rank int,
    aggregated_score float,
    teaching_learning_and_resources float,
    research_and_professional_practice float,
    graduation_outcomes float,
    outreach_and_inclusivity float,
    peer_perception float,
    PRIMARY KEY (faculty_id, year)
);